In [1]:
import os
import cv2
import numpy as np
from pylab import *
from PIL import Image
from scipy import ndimage
import glob
import h5py
from imutils import contours

# https://docs.opencv.org/3.4/d2/dbd/tutorial_distance_transform.html
from __future__ import print_function
import argparse
import random as rng

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D

import pickle

import copy

In [2]:
def hougher(img):
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    dst = cv2.Canny(gray, 50, 100)
    # change 100 to 200, so that diagonal isn't detected
    lines= cv2.HoughLines(dst, 1, math.pi/180.0, 180, np.array([]), 0, 0)
    a,b,c = lines.shape
    for i in range(a):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        a = math.cos(theta)
        b = math.sin(theta)
        x0, y0 = a*rho, b*rho
        pt1 = ( int(x0+1000*(-b)), int(y0+1000*(a)) )
        pt2 = ( int(x0-1000*(-b)), int(y0-1000*(a)) )
        cv2.line(img, pt1, pt2, (0, 0, 255), 7, cv2.LINE_AA)
    cv2.imwrite('houghed.jpg',img)

In [3]:
def sort_contours(cnts, method="left-to-right"):
    reverse = False
    i = 0
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
        key=lambda b:b[1][i], reverse=reverse))
    return (cnts, boundingBoxes)

In [4]:
def squares(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.medianBlur(gray, 5)
    sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    sharpen = cv2.filter2D(blur, -1, sharpen_kernel)

    thresh = cv2.threshold(sharpen,160,255, cv2.THRESH_BINARY_INV)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    close = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

    cnts = cv2.findContours(close, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    # (cnts, boundingBoxes) = sort_contours(cnts)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    img = cv2.drawContours(img, cnts, -1, (0,0,0), 3)
    cv2.imwrite("con.jpg", img)
    min_area = 4000
    max_area = 11000
    image_number = 0
    for c in cnts:
        area = cv2.contourArea(c)
        if area > min_area and area < max_area:
            x,y,w,h = cv2.boundingRect(c)
            ROI = img[y:y+h, x:x+w]
            cv2.imwrite('81/boxes_{}.jpg'.format(str(80-image_number).zfill(2)), ROI)
            cv2.rectangle(img, (x, y), (x + w, y + h), (0,0,0), 4)
            image_number += 1

In [5]:
file_path = 'gi.png'
img = cv2.imread(file_path)
img = cv2.resize(img, (900, 900))
hougher(img)
file_path = 'houghed.jpg'
img = cv2.imread(file_path)
img = cv2.resize(img, (900, 900))
squares(img)

In [6]:
def gray_all():
    image_number = 0
    for filepath in sorted(glob.iglob('81/*')):
        img = cv2.imread(filepath)
        img = cv2.resize(img, (28, 28))
        cv2.imwrite('81/boxes_{}.jpg'.format(str(80-image_number).zfill(2)), img)
        image_number += 1

In [7]:
# gray_all()

In [8]:
data_dir = "/home/vanillaskies/projects/computer-science/python/sudoku/training/"
categories = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]

In [9]:
training_data = []
def create_training_data():
    for category in categories:
        path = os.path.join(data_dir, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
                img = cv2.imread(os.path.join(path, img))
                training_data.append([img, class_num])

In [10]:
create_training_data()
print(len(training_data)) # should be number of images in training folder

2163


In [11]:
import random
random.shuffle(training_data)
X = []
y = []

In [12]:
for features, label in training_data:
    X.append(features)
    y.append(label)

In [13]:
X = np.array(X)
X=np.array(X/255.0)
y=np.array(y)

In [14]:
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [15]:
pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle", "rb")
y = pickle.load(pickle_in)

In [16]:
# X.reshape(list(X.shape) + [1])
model = Sequential()
# https://www.geeksforgeeks.org/keras-conv2d-class/
model.add(Conv2D(64, (5,5), input_shape = X.shape[1:]))
model.add(Activation("relu"))
# https://machinelearningmastery.com/pooling-layers-for-convolutional-neural-networks/
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))

model.add(Dense(32))
model.add(Activation('relu'))

model.add(Dense(16))
model.add(Activation('relu'))

model.add(Dense(10))
model.add(Activation('softmax'))

# https://gombru.github.io/2018/05/23/cross_entropy_loss/
# https://keras.io/api/losses/probabilistic_losses/#categoricalcrossentropy-class
# https://github.com/anujshah1003/own_data_cnn_implementation_keras/blob/master/custom_data_cnn.py

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=256, epochs=40, validation_split=0.3)

Epoch 1/40
6/6 [==============================] - 21s 3s/step - loss: 2.2677 - accuracy: 0.1521 - val_loss: 2.2026 - val_accuracy: 0.2419
Epoch 2/40
6/6 [==============================] - 10s 2s/step - loss: 2.1082 - accuracy: 0.2469 - val_loss: 1.9137 - val_accuracy: 0.2327
Epoch 3/40
6/6 [==============================] - 8s 1s/step - loss: 1.8642 - accuracy: 0.2691 - val_loss: 1.7222 - val_accuracy: 0.3914
Epoch 4/40
6/6 [==============================] - 9s 2s/step - loss: 1.6943 - accuracy: 0.3840 - val_loss: 1.5450 - val_accuracy: 0.4530
Epoch 5/40
6/6 [==============================] - 14s 2s/step - loss: 1.4862 - accuracy: 0.4842 - val_loss: 1.3360 - val_accuracy: 0.6148
Epoch 6/40
6/6 [==============================] - 9s 2s/step - loss: 1.2544 - accuracy: 0.6475 - val_loss: 1.1758 - val_accuracy: 0.6810
Epoch 7/40
6/6 [==============================] - 8s 1s/step - loss: 1.0588 - accuracy: 0.7112 - val_loss: 0.9418 - val_accuracy: 0.7319
Epoch 8/40
6/6 [======================

In [17]:
num_data_dir = "/home/vanillaskies/projects/computer-science/python/sudoku/81/"
def pic_to_sudoku(num_data_dir):
    sudoku = ""
    for img in sorted(os.listdir(num_data_dir)):
        print(os.path.join(num_data_dir, img))
        img = cv2.imread(os.path.join(num_data_dir, img))
        img = cv2.resize(img, (28, 28))
        test_image = np.array(img).astype('float32')
        test_image = np.expand_dims(test_image, axis=0)/255
        test_image = tf.image.resize_with_pad(test_image, 28, 28)
        f = model.predict(test_image)
        index = np.argmax(f)
        g = index.item()
        g = f'{g}'
        sudoku += g
    return sudoku

In [18]:
puzzle = pic_to_sudoku(num_data_dir)
puzzle

/home/vanillaskies/projects/computer-science/python/sudoku/81/boxes_00.jpg
/home/vanillaskies/projects/computer-science/python/sudoku/81/boxes_01.jpg
/home/vanillaskies/projects/computer-science/python/sudoku/81/boxes_02.jpg
/home/vanillaskies/projects/computer-science/python/sudoku/81/boxes_03.jpg
/home/vanillaskies/projects/computer-science/python/sudoku/81/boxes_04.jpg
/home/vanillaskies/projects/computer-science/python/sudoku/81/boxes_05.jpg
/home/vanillaskies/projects/computer-science/python/sudoku/81/boxes_06.jpg
/home/vanillaskies/projects/computer-science/python/sudoku/81/boxes_07.jpg
/home/vanillaskies/projects/computer-science/python/sudoku/81/boxes_08.jpg
/home/vanillaskies/projects/computer-science/python/sudoku/81/boxes_09.jpg
/home/vanillaskies/projects/computer-science/python/sudoku/81/boxes_10.jpg
/home/vanillaskies/projects/computer-science/python/sudoku/81/boxes_11.jpg
/home/vanillaskies/projects/computer-science/python/sudoku/81/boxes_12.jpg
/home/vanillaskies/projec

'489501020750000810000020594008090075500008000001003000160374082000005736003062450'

In [19]:
len(puzzle)

81

In [20]:
model.save('savehere')

INFO:tensorflow:Assets written to: savehere/assets


In [21]:
def hough_sudoku(img):
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    dst = cv2.Canny(gray, 50, 100)
    # change 100 to 200, so that diagonal isn't detected
    kernel = np.ones((5,5),np.uint8)
    erosion = cv2.erode(dst,kernel,iterations = 1)
    lines= cv2.HoughLines(dst, 1, math.pi/180.0, 180, np.array([]), 0, 0)
    a,b,c = lines.shape
    for i in range(a):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        a = math.cos(theta)
        b = math.sin(theta)
        x0, y0 = a*rho, b*rho
        pt1 = ( int(x0+1000*(-b)), int(y0+1000*(a)) )
        pt2 = ( int(x0-1000*(-b)), int(y0-1000*(a)) )
        cv2.line(img, pt1, pt2, (0, 0, 255), 7, cv2.LINE_AA)
    cv2.imwrite('find.jpg',img)

In [22]:
def find_sudoku(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.medianBlur(gray, 5)
    sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    sharpen = cv2.filter2D(blur, -1, sharpen_kernel)

    thresh = cv2.threshold(sharpen,160,255, cv2.THRESH_BINARY_INV)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    close = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

    cnts = cv2.findContours(close, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    img = cv2.drawContours(img, cnts, -1, (0,0,0), 3)
    # cv2.imwrite("con.jpg", img)
    min_area = 40500
    max_area = 810000
    image_number = 0
    
    for c in cnts:
        area = cv2.contourArea(c)
        if area > min_area and area < max_area:
            x,y,w,h = cv2.boundingRect(c)
            ROI = img[y:y+h, x:x+h]
            cv2.imwrite('cropped.jpg'.format(str(80-image_number).zfill(2)), ROI)
            cv2.rectangle(img, (x, y), (x + w, y + h), (36,255,12), 2)
            image_number += 1

In [23]:
file_path = '6.png'
img = cv2.imread(file_path)
# img = cv2.resize(img, (900, 900))
hough_sudoku(img)
file_path = 'find.jpg'
img = cv2.imread(file_path)
# img = cv2.resize(img, (900, 900))
find_sudoku(img)

In [24]:
def same_row(i,j): return (i/9 == j/9)
def same_col(i,j): return (i-j) % 9 == 0
def same_block(i,j): return (i/27 == j/27 and i%9/3 == j%9/3)

def r(a):
    printed = 0
    i = a.find('0')
    if i == -1 and printed == 0:
        print(a)
        printed = 1

    excluded_numbers = set()
    for j in range(81):
        if same_row(i,j) or same_col(i,j) or same_block(i,j):
          excluded_numbers.add(a[j])

    for m in '123456789':
        if m not in excluded_numbers:
            r(a[:i]+m+a[i+1:])

In [25]:
# r(puzzle)

In [26]:
def same_row(i,j): return (i/9 == j/9)
def same_col(i,j): return (i-j) % 9 == 0
def same_block(i,j): return (i/27 == j/27 and i%9/3 == j%9/3)

# function that solves a sudoku puzzle
#   Ref(s):
#   http://stackoverflow.com/questions/201461/shortest-sudoku-solver-in-python-how-does-it-work
def solve_puzzle(a):
    i = a.find('0')
    if i == -1:
        # puzzle is solved, format the output
        soln = []
        for j in range(81):
            soln.append(int(a[j]))
        print array(soln).reshape(9, 9)
        return
        
    # determine any excluded numbers
    excluded_numbers = set()
    for j in range(81):
        if same_row(i,j) or same_col(i,j) or same_block(i,j):
            excluded_numbers.add(a[j])

    for m in '123456789':
        if m not in excluded_numbers:
            # At this point, m is not excluded by any row, column, or block, so let's place it and recurse
            solve_puzzle(a[:i]+m+a[i+1:])

SyntaxError: invalid syntax (<ipython-input-26-d28c1dc1d589>, line 15)

In [ ]:
def splitBoxes(img):
    rows = np.vsplit(img, 9)
    boxes = []
    for r in rows:
        cols = np.hsplit(r, 9)
        for box in cols:
            boxes.append(box)
    return boxes

In [ ]:
file_path = '75.png'
img = cv2.imread(file_path)
img = cv2.resize(img, (900, 900))
hougher(img)
file_path = 'houghed.jpg'
img = cv2.imread(file_path)
boxes = splitBoxes(img)
plt.imshow(boxes[0][0])

In [27]:
puzzle

'489501020750000810000020594008090075500008000001003000160374082000005736003062450'